In [19]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img
import numpy as np
from PIL import ImageEnhance, Image, ImageFilter
import random
import cv2



# Assuming these directories are defined
original_data_dir = 'religeon-classification-task\\data\\trian\\chris'
augmented_data_dir = 'religeon-classification-task\\data\\trian\\augs-chris'
desired_augmented_images = 1

datagen = ImageDataGenerator(
    rotation_range=5, 
    zoom_range=0.05, 

)


def augment_image(img):
    img = img.convert('RGB')
    img = ImageEnhance.Brightness(img).enhance(random.uniform(0.4, 0.8))
    img = ImageEnhance.Contrast(img).enhance(random.uniform(0.5, 1.5))
    img = ImageEnhance.Color(img).enhance(random.uniform(0.5, 1.0))

    data = np.array(img)
    temp_factor = random.uniform(0.8, 1.2)
    data = data * [temp_factor, temp_factor, 1] if temp_factor > 1 else data * [1, 1, 1 / temp_factor]
    img = Image.fromarray(np.clip(data, 0, 255).astype(np.uint8))

    motion_type = random.choice(['horizontal', 'vertical', 'diagonal'])
    img = add_motion_blur(img, motion_type)

    return img

def add_motion_blur(img, direction='horizontal'):
    size = random.randint(2, 4)
    angle = random.randint(0, 360)

    if direction == 'horizontal':
        kernel = np.zeros((size, size))
        kernel[size // 2, :] = np.ones(size)
        kernel /= size
    elif direction == 'vertical':
        kernel = np.zeros((size, size))
        kernel[:, size // 2] = np.ones(size)
        kernel /= size
    elif direction == 'diagonal':
        kernel = np.zeros((size, size))
        np.fill_diagonal(kernel, 1)
        kernel /= size

    kernel = np.array(kernel)
    img = np.array(img)
    img = cv2.filter2D(img, -1, kernel)
    img = Image.fromarray(np.clip(img, 0, 255).astype(np.uint8))

    return img

for filename in os.listdir(original_data_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(original_data_dir, filename)
        img = load_img(img_path)
        img = augment_image(img)
        img_array = img_to_array(img)
        img_array = img_array.reshape((1,) + img_array.shape)  

        i = 0
        for batch in datagen.flow(img_array, 
                                  batch_size=1,  
                                  save_to_dir=augmented_data_dir, 
                                  save_prefix='aug_', 
                                  save_format='jpg'):
            augmented_image = batch[0].astype(np.uint8)
            save_img(os.path.join(augmented_data_dir, f'augmented_{i}_{filename}'), augmented_image)
            i += 1
            if i >= desired_augmented_images: 
                break
